In [74]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.bibframe.work import Work_Schema, Work_Edit, Work_Response

In [3]:
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

In [12]:
prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""

query = prefix+"""SELECT * WHERE { 
  graph work:bk-1 {
  work:bk-1 ?p ?o } }"""

response = acervoQuery.run_sparql(query)
# response = response.convert()
# bindings = response['results']['bindings']
# bindings

Type

In [29]:
queryType = prefix+"""SELECT * WHERE { 
  graph work:bk-1 {
  work:bk-1 rdf:type ?o } }"""
response = acervoQuery.run_sparql(queryType)
response = response.convert()
bindings = response['results']['bindings']

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'type': 'uri',
     'value': 'http://id.loc.gov/ontologies/bibframe/Work'}},
   {'o': {'type': 'uri',
     'value': 'http://id.loc.gov/ontologies/bibframe/Text'}}]}}

In [36]:
values = list()
for i in bindings:
    value = i['o']['value'].split("/")[-1]
    values.append(value)
print(values)

['Work', 'Text']


In [11]:
for i in bindings:
    metadado = i['p']['value']
    print(i['p']['value'])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://id.loc.gov/ontologies/bibframe/subject
http://id.loc.gov/ontologies/bibframe/subject
http://id.loc.gov/ontologies/bibframe/subject
http://id.loc.gov/ontologies/bibframe/contribution
http://id.loc.gov/ontologies/bibframe/language
http://id.loc.gov/ontologies/bibframe/adminMetadata
http://id.loc.gov/ontologies/bibframe/title
http://id.loc.gov/ontologies/bibframe/classification


Classification

In [37]:
query = """SELECT ?p ?o WHERE { graph work:bk-1 {
  work:bk-1  bf:classification ?classification .
    ?classification ?p ?o 
  } }"""

queryClassfication = prefix+query
response = acervoQuery.run_sparql(queryClassfication)
response = response.convert()
bindings = response['results']['bindings']
bindings

[{'p': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'o': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/ClassificationDdc'}},
 {'p': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/classificationPortion'},
  'o': {'type': 'literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
   'value': '584.6'}},
 {'p': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/itemPortion'},
  'o': {'type': 'literal', 'value': 'O48c'}}]

In [49]:
classification = {}
for i in bindings:
    metadadoUri = i['p']['value']
    if metadadoUri == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
        value = i['o']['value'].split('/')[-1]
        classification['type'] = value
    else:
        metadado = i['p']['value'].split('/')[-1]
        value = i['o']['value'].split('/')[-1]
        classification[metadado] = value

        print(metadado)

print(classification)

classificationPortion
itemPortion
{'type': 'ClassificationDdc', 'classificationPortion': '584.6', 'itemPortion': 'O48c'}


contribution

In [50]:
query = """SELECT ?p ?o WHERE { graph work:bk-1 {
  work:bk-1  bf:contribution ?contribution .
    ?contribution ?p ?o 
  } }"""

queryContribution = prefix+query
response = acervoQuery.run_sparql(queryContribution)
response = response.convert()
bindings = response['results']['bindings']
bindings

[{'p': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'},
  'o': {'type': 'literal', 'value': 'Karl R. Popper MUDADO'}},
 {'p': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'o': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/Contribution'}},
 {'p': {'type': 'uri', 'value': 'http://id.loc.gov/ontologies/bibframe/role'},
  'o': {'type': 'uri', 'value': 'http://id.loc.gov/vocabulary/relators/aut'}},
 {'p': {'type': 'uri', 'value': 'http://id.loc.gov/ontologies/bibframe/agent'},
  'o': {'type': 'uri',
   'value': 'https://bibliokeia.com/authorities/names/n80032184'}}]

In [73]:
contribution = {}
for i in bindings:
    metadadoUri = i['p']['value']
    if metadadoUri == 'http://www.w3.org/2000/01/rdf-schema#label':
        contribution['label'] = i['o']['value']
    elif metadadoUri == 'http://id.loc.gov/ontologies/bibframe/role':
        value = i['o']['value'].split('/')[-1]
        contribution['role'] = value
    elif metadadoUri == 'http://id.loc.gov/ontologies/bibframe/agent':
        value = i['o']['value']
        contribution['uri'] = value
        
print(contribution)

{'label': 'Karl R. Popper MUDADO', 'role': 'aut', 'uri': 'https://bibliokeia.com/authorities/names/n80032184'}


In [75]:
x = {
  "type": [
    "Work",
    "Text"
  ],
  "classification": {
    "type": "ClassificationDdc",
    "classificationPortion": "584.6",
    "itemPortion": "O48c"
  },
  "contribution": {
    "label": "Karl R. Popper MUDADO",
    "role": "aut",
    "uri": "https://bibliokeia.com/authorities/names/n80032184"
  }
}

r = Work_Response(**x)
r



ValidationError: 6 validation errors for Work_Response
classification -> label
  field required (type=value_error.missing)
classification -> role
  field required (type=value_error.missing)
classification -> uri
  field required (type=value_error.missing)
contribution -> type
  field required (type=value_error.missing)
contribution -> classificationPortion
  field required (type=value_error.missing)
contribution -> itemPortion
  field required (type=value_error.missing)